In [8]:
import pandas as pd
import numpy as np
import json
from bardapi import Bard
import os

from translate import Translator
from tqdm.auto import tqdm
import re
from difflib import SequenceMatcher
import spacy

from bs4 import BeautifulSoup
import requests
import urllib.request
import time

from tqdm.auto import tqdm

import spacy
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

In [9]:
from processing import translate_dataset, split_data, prepare_prompt, prep_fewshot
from deep_translator import GoogleTranslator

ModuleNotFoundError: No module named 'processing'

## translating the test data to use some of it in the prompt

In [11]:
os.environ['_BARD_API_KEY'] = "API KEY"

In [12]:
with open('Testdata.json') as f:
    data = json.load(f)

# Translate the dataset
translated_data = translate_dataset(data)

FileNotFoundError: [Errno 2] No such file or directory: 'Testdata.json'

In [254]:
print(translated_data)

[{'text': "I am looking for a Tissot brand men's watch with a price below 1000DT.", 'entities': [{'entity': 'Price', 'value': 'Less than 1000DT'}]}, {'text': 'I want to buy Ray-Ban brand sunglasses with a price between 9 and 1420DT.', 'entities': [{'entity': 'Price', 'value': 'Between 9 and 1420DT'}]}, {'text': "I am looking for a women's perfume from the Chanel brand with a volume of 50ml.", 'entities': [{'entity': 'Volume', 'value': '50ml'}]}, {'text': 'I want to find a JEEP brand bag with a price lower than 199DT.', 'entities': [{'entity': 'Price', 'value': 'Less than 199TND'}]}, {'text': "I want to buy Stephan brand men's shoes with a shoe size of 43.", 'entities': [{'entity': 'Shoe size', 'value': '43'}]}, {'text': 'I am looking for jewelry from the Yves Saint Laurent brand with a price between 15 and 1588DT.', 'entities': [{'entity': 'Price', 'value': 'Between 15 and 1588DT'}]}, {'text': 'I want to buy a Clarins brand anti-aging care with a volume of 50ml.', 'entities': [{'entity

In [265]:
translatedKeys = ["Category", "subcategory", "Brand", "Price", "volume","shoe size"]

In [266]:
def split_data(data):
    np.random.shuffle(data)
    spl = int(len(data) * 0.2)
    print(spl)
    prompt = data[:spl]
    eval = data[spl:]
    return prompt, eval

In [267]:
prompt, eval = split_data(translated_data)

3


## few shot prompt prep

In [268]:
#prepare prompt
prompt = prepare_prompt(prompt, translatedKeys)

[{'text': 'I want to buy a Clarins brand anti-aging care with a volume of 50ml.', 'entities': [{'entity': 'Category', 'value': 'Care'}, {'entity': 'Subcategory', 'value': 'Anti-aging'}, {'entity': 'Brand', 'value': 'Clarins'}, {'entity': 'Volume', 'value': '50ml'}]}, {'text': 'I want to buy a Clarins brand anti-aging care with a volume of 50ml.', 'entities': [{'entity': 'Category', 'value': 'Care'}, {'entity': 'Subcategory', 'value': 'Anti-aging'}, {'entity': 'Brand', 'value': 'Clarins'}, {'entity': 'Volume', 'value': '50ml'}]}, {'text': 'I want to find a JEEP brand bag with a price lower than 199DT.', 'entities': [{'entity': 'Category', 'value': 'bag'}, {'entity': 'Brand', 'value': 'JEEP'}, {'entity': 'Price', 'value': 'Less than 199TND'}]}]


In [269]:
print("prompt: ", prompt)

prompt:  "Extract the relevant information from the following user query, these are the possible categories: 
 ['Category', 'subcategory', 'Brand', 'Price', 'volume', 'shoe size']
 please just give me the answer for the last query only. give me the answer directly by following the pattern bellow, and make sure to follow the same writing pattern 
here are some examples of the query and the relevant information to extract: 

Query: I want to buy a Clarins brand anti-aging care with a volume of 50ml.
**Category** Care
**Subcategory** Anti-aging
**Brand** Clarins
**Volume** 50ml

Query: I want to buy a Clarins brand anti-aging care with a volume of 50ml.
**Category** Care
**Subcategory** Anti-aging
**Brand** Clarins
**Volume** 50ml

Query: I want to find a JEEP brand bag with a price lower than 199DT.
**Category** bag
**Brand** JEEP
**Price** Less than 199TND



In [165]:
prep = prep_fewshot(prompt, eval[5])
print(prep)

"Extract the relevant information from the following user query, these are the possible categories: 
 ['Category', 'subcategory', 'Brand', 'Price', 'volume', 'shoe size']
 please just give me the answer for the last query only. give me the answer directly by following the pattern bellow, and make sure to follow the same writing pattern 
here are some examples of the query and the relevant information to extract: 

Query: I am looking for a men's watch from Tissot with a price below 1000DT.
**Category** shows
**SUBCATEGORY** men's watch
**Brand** Tissot
**Price** Less than 1000DT

Query: I want to buy Ray-Ban brand sunglasses with a price between 9 and 1420DT.
**Category** eyewear
**SUBCATEGORY** Sunglasses
**Brand** Rayban
**Price** Between 9 and 1420DT

Query: I am looking for a perfume for women from Chanel with a volume of 50ml.
**Category** perfume
**SUBCATEGORY** perfume for women
**Brand** Chanel
**Volume** 50 ml

Query: I am looking for a men's watch from Tissot with a price bel

In [166]:
response = Bard().get_answer(prep)

In [167]:
print(response['content'])

Sure, here is the relevant information extracted from the last query:

**Category:** watches
**Subcategory:** men's watch
**Brand:** Tissot
**Price:** below 1000DT


## extracting the necessary entities from the response of bard

In [168]:
def extract_entities(response):
    entities = []
    responseSplited = response['content'].split("\n")
    responseFiltered = []
    for index,ele in enumerate(responseSplited):
        if "**" in ele:
            responseFiltered.append(ele)
    responseFiltered = list(filter(None, responseFiltered))
    for index,ele in enumerate(responseFiltered):
        elements = ele.split("**")
        entities.append({
            'entity': elements[1].strip(),
            'value': elements[-1].strip()
        })
    return entities

extracted_entities = extract_entities(response)

In [169]:
extracted_entities

[{'entity': 'Category:', 'value': 'watches'},
 {'entity': 'Subcategory:', 'value': "men's watch"},
 {'entity': 'Brand:', 'value': 'Tissot'},
 {'entity': 'Price:', 'value': 'below 1000DT'}]

## extracting the value relation from the user query, "whether the user wants the value of the product to be higher lower or between values"

In [170]:
def extract_value_relation(value_string):
    pattern = r"\d+\.?\d*"  # Regular expression pattern to match numbers
    numbers = re.findall(pattern, value_string)
    numbers = [int(num) for num in numbers]  
    print(numbers)    
    if len(numbers) == 2:
        return "between", numbers
    pattern = r"\b(greater|more|less|below|bellow|between)\b"

    matches = re.search(pattern, value_string, re.IGNORECASE)

    if matches:
        relation = matches.group(1)
        
        if relation.strip().lower() in ["greater", "more"]:
            # Logic for greater or more
            value1 = re.findall(r"[\d.,]+", value_string)
            return f"greater", value1[0] if value1 else None
        elif relation.strip().lower() in ["less", "below", "bellow"]:
            # Logic for less, below, or bellow
            value1 = re.findall(r"[\d.,]+", value_string)
            return f"less", value1[0] if value1 else None
        elif relation.strip().lower() == "between":
            # Logic for between
            values = re.findall(r"[\d.,]+", value_string)
            return f"between", values[:2] if len(values) >= 2 else None
        else:
            return "Invalid value relation.", 0
        
    else:
        return "No value relation found.", 0

In [171]:
def extract_price_max_min(entities):
    for i in entities:
        if "price" in i['entity'].lower():
            return extract_value_relation(i['value'])

In [172]:
symb, vall = extract_price_max_min(extracted_entities)

[1000]


In [173]:
print(vall)

1000


## preparing the query to search for the products

In [174]:
def prepare_web_search_query(entities):
    query = ""
    translator = Translator(from_lang="en", to_lang="fr")
    for i in entities:
        if "price" not in i['entity'].lower():
            query += f"{translator.translate(i['value'])} "
    return query

In [175]:
prepare_web_search_query(extracted_entities)

'montres Montre pour homme Tissot '

In [176]:
def get_recommended_articles(entities):
    val = len(entities)
    while val>0:
        query = prepare_web_search_query(entities[:val])
        query = query.replace(" ", "+")
        url = f"https://www.lamode.tn/recherche?controller=search&s={query}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        try:
            articles = soup.find("div", class_="products").find_all("article")
            links = []
            for article in articles:
                links.append(article.find("a")['href'])
            return links
        except:
            val = val-1
            continue
    return []

In [177]:
li = get_recommended_articles(extracted_entities)
print(li)

['https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/560-montre-homme-tissot-t035-617-16-051-00.html', 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/2398-montre-homme-tissot-t063-617-36-037-00.html', 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5145-montre-homme-tissot-t120-417-11-041-00.html', 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/1577-montre-homme-tissot-t035-617-11-031-00.html', 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/1722-montre-homme-tissot-t100-417-16-051-00.html', 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5147-montre-homme-tissot-t120-417-11-051-00.html', 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/2702-montre-homme-tissot-t055-417-16-057-00.html', 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/381

In [178]:
len(li)

20

## extracting the product descriptions from the products if it is present

In [179]:
def get_articles_descriptions(links):
    descriptions = []
    for link in tqdm(links):
        response = requests.get(link)
        soup = BeautifulSoup(response.text, "html.parser")
        try:
            descriptions.append((link,soup.find("div", class_="product-description").text))
        except:
            try:
                descriptions.append((link,soup.find("div", class_="tvproduct-page-decs").text))
            except:
                descriptions.append((link,"None"))
    return descriptions

In [180]:
descs = get_articles_descriptions(li)

100%|██████████| 20/20 [00:25<00:00,  1.27s/it]


In [181]:
print(symb)
print(vall)

less
1000


## filter the products based on the price

In [203]:
def filter_based_on_price(links, symb, vall):
    filtered = []    
    for link in links:
        response = requests.get(link)
        soup = BeautifulSoup(response.text, "html.parser")
        prr = soup.find("span", class_="price")['content']
        if symb.strip().lower() == "between":
            if float(vall[0]) <= float(prr) <= float(vall[1]):
                filtered.append(link)
        elif symb.strip().lower() == "greater":
            if float(prr) >= float(vall):
                filtered.append(link)
        elif symb.strip().lower() == "less":
            if float(prr) <= float(vall):
                filtered.append(link)
    return filtered

In [187]:
print(vall)

1000


In [188]:
type(vall)

str

In [204]:
filtered_descs = filter_based_on_price(li, symb, vall)
filtered_descs

['https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/2702-montre-homme-tissot-t055-417-16-057-00.html',
 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/3021-montre-homme-tissot-t055-417-11-037-00.html',
 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/3330-montre-homme-tissot-t055-417-16-037-00.html',
 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5143-montre-homme-tissot-t122-417-16-011-00.html',
 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5144-montre-homme-tissot-t122-417-16-051-00.html',
 'https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/14122-bracelet-de-montres-tissot-t120-417bs.html']

In [201]:
print(len(filtered_descs))

6


## ranking the most relevant products based on the product description with 2 different models

In [76]:
def compare_article_query_SentenceTransformer(query, article):
    model = SentenceTransformer("dangvantuan/sentence-camembert-base")
    query_embedding = model.encode(query)
    article_embeddings = model.encode(article)
    cos_scores = util.pytorch_cos_sim(query_embedding, article_embeddings)[0]
    cos_scores = cos_scores.numpy()[0]
    return cos_scores

In [62]:
def compare_article_query_Spacy(query, article, pipeline="fr_core_news_sm"):
    nlp = spacy.load(pipeline)
    query_doc = nlp(query)
    article_doc = nlp(article)
    return query_doc.similarity(article_doc)

In [78]:
def get_recommended_articles_spacy(query, descs, pipeline="fr_core_news_sm"):
    recommended_articles = []
    for link, desc in tqdm(descs):
        score = compare_article_query_Spacy(query, desc, pipeline)
        recommended_articles.append((link, score))
    recommended_articles.sort(key=lambda x: x[1], reverse=True)
    return recommended_articles

In [81]:
def get_recommended_articles_sentence_transformer(query, descs):
    recommended_articles = []
    for link, desc in tqdm(descs):
        score = compare_article_query_SentenceTransformer(query, desc)
        recommended_articles.append((link, score))
    recommended_articles.sort(key=lambda x: x[1], reverse=True)
    return recommended_articles

In [79]:
rec = get_recommended_articles_spacy("montre homme tissot", descs)

  0%|          | 0/20 [00:00<?, ?it/s]C:\Users\Hamadi\AppData\Local\Temp\ipykernel_16628\2505155689.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  return query_doc.similarity(article_doc)
100%|██████████| 20/20 [00:33<00:00,  1.67s/it]


In [82]:
rec1 = get_recommended_articles_sentence_transformer("montre homme tissot", descs)

100%|██████████| 20/20 [00:36<00:00,  1.84s/it]


In [83]:
print(rec)
print(rec1)

[('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5145-montre-homme-tissot-t120-417-11-041-00.html', 0.3930250078638776), ('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/2398-montre-homme-tissot-t063-617-36-037-00.html', 0.3817942141675934), ('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/560-montre-homme-tissot-t035-617-16-051-00.html', 0.2740323911776584), ('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5147-montre-homme-tissot-t120-417-11-051-00.html', 0.21352381000931075), ('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/14122-bracelet-de-montres-tissot-t120-417bs.html', 0.17154669130452652), ('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/1722-montre-homme-tissot-t100-417-16-051-00.html', 0.10360418719016935), ('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisi

In [176]:
descs

[('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/560-montre-homme-tissot-t035-617-16-051-00.html',
  " La Tissot T035.617.16.051.00 est une montre pour hommes fonctionnelle et attrayante de la \ncollection Chrono XL. Le boîtier est en acier inoxydable, ce qui garantit la haute qualité de l'article, \ntandis que le cadran est de couleur bleue. Les caractéristiques de la montre incluent (entre autres) \nun chronographe. \nL'étanchéité à 100 mètres protège la montre et lui permet d'être immergée dans l'eau pour un usage \nquotidien, y compris la natation, mais pas pour les sports nautiques à fort impact. Nous l'expédions \navec une boîte originale et une garantie du fabricant. "),
 ('https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/2398-montre-homme-tissot-t063-617-36-037-00.html',
  " La famille Tissot Tradition apporte une touche de nostalgie à l'horlogerie moderne, en agrémentant \nles technologies les plus innovantes d'un 

In [217]:
for i, (link, score) in enumerate(rec[:5]):
        print(f"{i+1}. {link} - {score}")

1. https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5145-montre-homme-tissot-t120-417-11-041-00.html - 0.3930250078638776
2. https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/2398-montre-homme-tissot-t063-617-36-037-00.html - 0.3817942141675934
3. https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/560-montre-homme-tissot-t035-617-16-051-00.html - 0.2740323911776584
4. https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/5147-montre-homme-tissot-t120-417-11-051-00.html - 0.21352381000931075
5. https://www.lamode.tn/montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie/14122-bracelet-de-montres-tissot-t120-417bs.html - 0.17154669130452652
